In [7]:


import cv2
import os
import numpy as np
import random

def load_and_preprocess_dataset(dataset_path, target_size=(64, 64)):
    image_data = []
    labels = []

    for dirs in os.listdir(dataset_path):
        
        for dir in dirs:
            
            a=os.path.join(dataset_path, dir)
            
            for root1,dirs1,files1 in os.walk(a):
                for file in files1:
                    
                    if file.endswith(".jpg"):
                        image_path = os.path.join(a, file)
                        label = dir
                        
                        

                        # Load and preprocess the image
                        img = cv2.imread(image_path)
                        img = cv2.resize(img, target_size)
                        img = img.astype(np.float32) / 255.0

                        image_data.append(img)
                        labels.append(label)
            
    # return 0,0
    return np.array(image_data), np.array(labels)

dataset_path =  r'C:\Users\HP\Downloads\lfw\train'
x_train, y_train = load_and_preprocess_dataset(dataset_path)
print (x_train[0:5])
print (y_train[0:5])





[[[[0.41960785 0.627451   0.7490196 ]
   [0.41568628 0.627451   0.75686276]
   [0.39607844 0.6117647  0.7529412 ]
   ...
   [0.6745098  0.8392157  0.8666667 ]
   [0.6784314  0.8352941  0.8666667 ]
   [0.6745098  0.81960785 0.8509804 ]]

  [[0.4117647  0.6392157  0.8       ]
   [0.38039216 0.6156863  0.76862746]
   [0.3882353  0.6117647  0.7529412 ]
   ...
   [0.68235296 0.84705883 0.8784314 ]
   [0.68235296 0.8392157  0.87058824]
   [0.6745098  0.81960785 0.8509804 ]]

  [[0.38039216 0.60784316 0.78431374]
   [0.3137255  0.54901963 0.70980394]
   [0.33333334 0.5568628  0.69803923]
   ...
   [0.6745098  0.84313726 0.87058824]
   [0.68235296 0.8392157  0.87058824]
   [0.6862745  0.83137256 0.8627451 ]]

  ...

  [[0.12156863 0.07450981 0.10980392]
   [0.13333334 0.09803922 0.13725491]
   [0.14509805 0.10588235 0.15294118]
   ...
   [0.7058824  0.79607844 0.827451  ]
   [0.7019608  0.7921569  0.8235294 ]
   [0.69411767 0.78431374 0.8156863 ]]

  [[0.1254902  0.08235294 0.11764706]
   [0.1

In [8]:
import numpy as np

# Assuming you have loaded your x_train and y_train data from your dataset
# If not, make sure to load them first

# Get the number of samples in your training data
num_samples = x_train.shape[0]

# Create a random permutation of indices to shuffle the data
permutation = np.random.permutation(num_samples)

# Use the permutation to shuffle both x_train and y_train
x_train = x_train[permutation]
y_train = y_train[permutation]


In [10]:
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Input, Dense, Lambda, Flatten
from tensorflow.keras.models import Model
from sklearn.metrics import confusion_matrix, classification_report
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.datasets import fetch_lfw_people
from sklearn.model_selection import train_test_split

from keras.models import Sequential
# include convolutional layers
from keras.layers import Conv2D, Dropout
# Pooling layers
from keras.layers import MaxPooling2D
# flatten layers into single vector
from keras.layers import Flatten
from keras.layers import Dense

In [12]:




def cosine_loss(y_true, y_pred):
    scale = 8.0
    margin = 0.20
    t=0.8

    # Convert y_true to float32
    y_true = tf.cast(y_true, tf.float32)

    # L2 normalize embeddings
    costheta = K.l2_normalize(y_pred, axis=1)
    theta=tf.acos(costheta)
    # print(y_pred.shape)
    m_theta=theta+margin*y_true
    m_costheta=tf.cos(m_theta)
    temp=tf.reduce_sum((y_true*m_costheta),axis=1)
    extended_tensor = tf.expand_dims(temp, axis=1)
    # Duplicate the original data 7 times
    value = tf.concat([extended_tensor] * num_classes, axis=1)
    # print(value.shape)

    condition=m_costheta>value
    f=tf.where(condition,t*(m_costheta),m_costheta)
    # print(f.shape)
    y_pred=scale*f

    # Compute the final Cosine loss
    loss = K.categorical_crossentropy(y_true, y_pred, from_logits=True)
    # Combine the loss with the angular margin penalty
    cos_loss = K.mean(loss , axis=-1)

    return cos_loss


# def cosine_loss(y_true, y_pred):
#     scale = 30.0
#     margin = 0.35


#     # Convert y_true to float32
#     y_true = tf.cast(y_true, tf.float32)

#     # L2 normalize embeddings
#     costheta = K.l2_normalize(y_pred, axis=1)
#     theta=tf.acos(costheta)

#     m_theta=theta+margin*y_true
#     m_costheta=tf.cos(m_theta)

#     y_pred=scale*m_costheta

#     # Compute the final Cosine loss
#     loss = K.categorical_crossentropy(y_true, y_pred, from_logits=True)
#     # Combine the loss with the angular margin penalty
#     cos_loss = K.mean(loss , axis=-1)

#     return cos_loss






In [13]:
print(y_train[:10])

['Gerhard_Schroeder' 'Ariel_Sharon' 'Colin_Powell' 'Colin_Powell'
 'George_W_Bush' 'Tony_Blair' 'Colin_Powell' 'George_W_Bush' 'Tony_Blair'
 'George_W_Bush']


In [14]:
# Split the dataset into training and testing sets
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.1, random_state=42)

# Preprocess the data, e.g., resize images to a common size and normalize pixel values
x_train = [cv2.resize(image, (64, 64)) for image in x_train]
x_val = [cv2.resize(image, (64, 64)) for image in x_val]

# Determine the number of classes
num_classes = len(set(y_train))
print(num_classes)

x_test, y_test = x_val, y_val

6


In [8]:
from sklearn.preprocessing import LabelEncoder


import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Activation
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import accuracy_score
from tensorflow.keras.utils import to_categorical

label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)

num_classes = len(np.unique(y_train_encoded))
labels_one_hot = to_categorical(y_train_encoded, num_classes=num_classes)

# print(labels_one_hot)

x_train, x_test, y_train, y_test = train_test_split(x_train, labels_one_hot, test_size=0.2, random_state=42)



ValueError: y should be a 1d array, got an array of shape (310, 3) instead.

In [3]:
def cosface_loss(y_true, y_pred):
    scale = 30.0
    margin = 0.35

    # Convert y_true to float32
    y_true = tf.cast(y_true, tf.float32)

    # L2 normalize embeddings
    y_pred = K.l2_normalize(y_pred, axis=1)

    y_pred=scale*(y_pred-margin*y_true)

    # Compute the final CosFace loss
    loss = K.categorical_crossentropy(y_true, y_pred, from_logits=True)


    # Combine the loss with the angular margin penalty
    cos_loss = K.mean(loss , axis=-1)

    return cos_loss

In [5]:


# class CosFaceLoss(tf.keras.losses.Loss):
#     def __init__(self, margin=0.35, scale=30):
#         super(CosFaceLoss, self).__init__()
#         self.margin = margin
#         self.scale = scale

#     def call(self, y_true, y_pred):
#         y_pred = tf.nn.l2_normalize(y_pred, axis=1)
#         cos_t = tf.reduce_sum(y_pred * y_true, axis=1)
#         cos_m = tf.math.cos(self.margin)
        
#         # Calculate the margin loss only for positive samples (where y_true has a positive class)
#         positive_samples = tf.math.reduce_any(y_true > 0, axis=1)
#         positive_cos_t = tf.boolean_mask(cos_t, positive_samples)
#         loss = tf.reduce_mean(tf.where(positive_samples, tf.math.acos(cos_m) - positive_cos_t, 0.0))
        
#         return loss * self.scale


In [4]:
print(y_train.shape)
print(x_train.shape)
print(num_classes)

(310, 3)
(310, 128, 128, 3)
3


In [5]:
from tensorflow.keras.layers import Input, Flatten, Dense, Dropout, Lambda
from tensorflow.keras import backend as K
# Define the model architecture
def create_model(num_classes):
    
    input_tensor = Input(shape=(128, 128, 3))
    x = Conv2D(64, (3, 3), activation='relu')(input_tensor)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    x = Conv2D(128, (3, 3), activation='relu')(x)
    x = MaxPooling2D((2, 2))(x)
    x = BatchNormalization()(x)
    x = Flatten()(input_tensor)
    x = Dense(512, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = BatchNormalization()(x)
    x = Activation('leakyrelu')(x)
    x = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs=input_tensor, outputs=x)
    return model

# Create and compile the model
model = create_model(num_classes=num_classes)
model.compile(optimizer=Adam(), loss=cosface_loss, metrics=['accuracy'])

# Training the model
model.fit(x_train, y_train, epochs=10, batch_size=32, validation_data=(x_test, y_test))

# Evaluate the model
loss, accuracy = model.evaluate(x_test, y_test)
print(f"Test loss: {loss:.4f}, Test accuracy: {accuracy:.4f}")

# Save the model and label encoder
model.save("face_recognition_model.h5")
np.save("label_encoder.npy", 5749)


Epoch 1/10
10/10 [==============================] - 8s 558ms/step - loss: 18.1440 - accuracy: 0.3742 - val_loss: 19.9062 - val_accuracy: 0.4359
Epoch 2/10
10/10 [==============================] - 5s 486ms/step - loss: 14.1970 - accuracy: 0.4806 - val_loss: 16.5140 - val_accuracy: 0.5000
Epoch 3/10
10/10 [==============================] - 5s 477ms/step - loss: 13.3357 - accuracy: 0.5032 - val_loss: 21.5702 - val_accuracy: 0.3718
Epoch 4/10
10/10 [==============================] - 5s 478ms/step - loss: 11.8205 - accuracy: 0.5645 - val_loss: 14.9528 - val_accuracy: 0.5000
Epoch 5/10
10/10 [==============================] - 5s 479ms/step - loss: 10.9142 - accuracy: 0.5871 - val_loss: 13.1535 - val_accuracy: 0.5256
Epoch 6/10
10/10 [==============================] - 5s 503ms/step - loss: 11.1274 - accuracy: 0.5677 - val_loss: 12.7449 - val_accuracy: 0.5256
Epoch 7/10
10/10 [==============================] - 5s 499ms/step - loss: 9.3596 - accuracy: 0.6258 - val_loss: 16.0371 - val_accuracy: 

c:\Users\HP\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [16]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)  # Add validation split

batch_size = 20

train_generator = train_datagen.flow_from_directory(
    r'C:\Users\HP\Downloads\lfw\train',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',  # 'binary' for binary classification
    subset='training'     # Specify subset as 'training' for training data
)

validation_generator = train_datagen.flow_from_directory(
    r'C:\Users\HP\Downloads\lfw\train',
    target_size=(150, 150),
    batch_size=batch_size,
    class_mode='categorical',  # 'binary' for binary classification
    subset='validation'   # Specify subset as 'validation' for validation data
)

Found 976 images belonging to 6 classes.
Found 241 images belonging to 6 classes.


In [ ]:
print(train_generator.shape)

(array([[[[0.02745098, 0.        , 0.        ],
         [0.03529412, 0.        , 0.        ],
         [0.03137255, 0.        , 0.        ],
         ...,
         [0.        , 0.        , 0.07058824],
         [0.        , 0.        , 0.0627451 ],
         [0.        , 0.        , 0.0627451 ]],

        [[0.6039216 , 0.56078434, 0.47450984],
         [0.58431375, 0.54509807, 0.44705886],
         [0.5411765 , 0.49803925, 0.38823533],
         ...,
         [0.7960785 , 0.83921576, 0.9176471 ],
         [0.78823537, 0.8313726 , 0.90196085],
         [0.78823537, 0.8352942 , 0.8980393 ]],

        [[0.6039216 , 0.54901963, 0.41176474],
         [0.59607846, 0.5411765 , 0.39607847],
         [0.5529412 , 0.49411768, 0.34117648],
         ...,
         [0.7843138 , 0.8470589 , 0.9450981 ],
         [0.7843138 , 0.8431373 , 0.9333334 ],
         [0.7803922 , 0.83921576, 0.9294118 ]],

        ...,

        [[0.0627451 , 0.0509804 , 0.01568628],
         [0.05490196, 0.04313726, 0.01568628

In [26]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(150, 150, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(6, activation=None)  # Binary classification output
])

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

history = model.fit(train_generator, epochs=10, validation_data=validation_generator)

Epoch 1/10
49/49 [==============================] - 42s 836ms/step - loss: 11.1282 - accuracy: 0.2449 - val_loss: 12.8410 - val_accuracy: 0.1950
Epoch 2/10
49/49 [==============================] - 41s 826ms/step - loss: 12.7492 - accuracy: 0.1936 - val_loss: 12.8410 - val_accuracy: 0.1950
Epoch 3/10
49/49 [==============================] - 42s 848ms/step - loss: 12.7492 - accuracy: 0.1936 - val_loss: 12.8410 - val_accuracy: 0.1950
Epoch 4/10
49/49 [==============================] - 41s 837ms/step - loss: 12.7492 - accuracy: 0.1936 - val_loss: 12.8410 - val_accuracy: 0.1950
Epoch 5/10
49/49 [==============================] - 42s 866ms/step - loss: 12.7492 - accuracy: 0.1936 - val_loss: 12.8410 - val_accuracy: 0.1950
Epoch 6/10
28/49 [================>.............] - ETA: 21s - loss: 12.4915 - accuracy: 0.1857